In [1]:
import numpy as np
import pandas as pd
from matplotlib.patches import ConnectionPatch
import scipy.spatial.distance as dist

In [2]:

#Implementation borrowed and modified from "https://github.com/kamperh/lecture_dtw_notebook/blob/main/dtw.ipynb"

class Dtw:
    def __init__(self,x=None,y=None):
        if x is None or y is  None:
            raise ValueError("You should enter proper value of x and y")
        else:
            self.x=x
            self.y=y
            
        self.alignment_cost= self.get_alignment_cost(self.x,self.y)

    # Function 'dp' is 
    def dp(self,dist_mat):
        N, M = dist_mat.shape
        
        cost_mat = np.zeros((N + 1, M + 1))
        for i in range(1, N + 1):
            cost_mat[i, 0] = np.inf
        for i in range(1, M + 1):
            cost_mat[0, i] = np.inf
        traceback_mat = np.zeros((N, M))
        for i in range(N):
            for j in range(M):
                penalty = [
                    cost_mat[i, j],      # match (0)
                    cost_mat[i, j + 1],  # insertion (1)
                    cost_mat[i + 1, j]]  # deletion (2)
                i_penalty = np.argmin(penalty)
                cost_mat[i + 1, j + 1] = dist_mat[i, j] + penalty[i_penalty]
                traceback_mat[i, j] = i_penalty
    
        i = N - 1
        j = M - 1
        path = [(i, j)]
        while i > 0 or j > 0:
            tb_type = traceback_mat[i, j]
            if tb_type == 0:
                i = i - 1
                j = j - 1
            elif tb_type == 1:
                i = i - 1
            elif tb_type == 2:
                j = j - 1
            path.append((i, j))
        cost_mat = cost_mat[1:, 1:]
        return (path[::-1], cost_mat)


    def get_alignment_cost(self,x,y):
        N = x.shape[0]
        M = y.shape[0]
        dist_mat = np.zeros((N, M))
        for i in range(N):
            for j in range(M):
                dist_mat[i, j] = abs(x[i] - y[j])
        
        path, cost_mat = self.dp(dist_mat)
        
        return "{:.4f}".format(cost_mat[N - 1, M - 1]/(N + M))
